In [1]:
import numpy as np
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import json
import os

In [2]:
#loaing Data
train_df = pd.read_csv('train.csv') 
test_df = pd.read_csv('test.csv')

In [3]:
#dictionary which contain responce code for each feature
with open('responce_dic.json', 'r') as fp:
    responce_dic = json.load(fp)

In [4]:
def final_fun_1(X):
    test_df_fun = X.copy()
    test_df_fun = test_df.iloc[:,1:]
    predictions = np.zeros(test_df_fun.shape[0])
    for i in test_df_fun.columns:
        test_df_fun[i+'_r'] = test_df_fun[i].map(responce_dic[i])
    for model in os.listdir('models'):
        clf = lgb.Booster(model_file='models/'+model)
        predictions += clf.predict(test_df_fun, num_iteration=clf.best_iteration) / 7
    
    predictions_out= []
    for i in predictions:
        if i>=0.095:
            predictions_out.append(1)
        else:
            predictions_out.append(0)
    return np.array(predictions_out)

In [5]:
final_fun_1(test_df.iloc[:1,:])

array([0, 1, 0, ..., 0, 0, 0])

In [6]:
def final_fun_2(X,Y):
    
    train_fun = X.copy()
    train_fun = train_fun.iloc[:,1:]
    predictions = np.zeros(train_fun.shape[0])
    for i in train_fun.columns:
        train_fun[i+'_r'] = train_fun[i].map(responce_dic[i])
    for model in os.listdir('models'):
        clf = lgb.Booster(model_file='models/'+model)
        predictions += clf.predict(train_fun, num_iteration=clf.best_iteration) / 7
    
    return roc_auc_score(Y, predictions)

In [65]:
from sklearn.metrics import roc_auc_score
final_fun_2(train_df.drop('target',axis = 1),train_df['target'])

0.9458312999944145

In [ ]:
#if needed return function 2 with predictions
from sklearn.metrics import confusion_matrix
from seaborn import heatmap
import matplotlib.pyplot as plt
heatmap(confusion_matrix(train_df['target'],predictions ),annot=True, fmt="d",xticklabels=["Pred:NO","Pred:YES"],yticklabels=["Actual:NO","Actual:YES"])
plt.title("Confusion matrix for Train data")